In [1]:
import ee
ee.Initialize()

In [ ]:
import numpy as np
dateFilter = ee.Filter.date('2014-01-01', '2014-12-31')
modis_sr=ee.ImageCollection('MODIS/006/MOD09GA').select('sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07').filter(dateFilter)
modis_lst=ee.ImageCollection('MODIS/006/MOD11A2').select('LST_Day_1km').filter(dateFilter)
modis_ndvi=ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filter(dateFilter)
filterTimeEq = ee.Filter.equals( leftField= 'system:time_start', rightField= 'system:time_start' ) 
simpleJoin = ee.Join.simple() 
srjoin = simpleJoin.apply(modis_sr, modis_lst, filterTimeEq)
print(srjoin.size().getInfo())
sr=srjoin.getInfo()['features']
index=len(sr)
lstjoin= simpleJoin.apply(modis_lst,modis_sr,filterTimeEq)
lst=lstjoin.getInfo()['features']
ndvijoin=simpleJoin.apply(modis_ndvi,lstjoin,filterTimeEq)
ndvi=ndvijoin.getInfo()['features']
region = ee.Feature(ee.FeatureCollection('ft:1Ec8IWsP8asxN-ywSqgXWMuBaxI6pPaeh6hC64lA').filter(ee.Filter.eq('G200_REGIO', 'Sierra Nevada Coniferous Forests')).first())
flag=0
hist3d=[]
for i in range(index):
  sr_img=ee.Image(sr[i]["id"]).select('sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07')
  lst_img=ee.Image(lst[i]['id']).select('LST_Day_1km','LST_Night_1km')
  ndvi_img=ee.Image(ndvi[i]['id'])
  sr_img=sr_img.updateMask(ndvi_img.select('NDVI').lte(0.55) and ndvi_img.select('NDVI').gte(0.1))
  lst_img=lst_img.updateMask(ndvi_img.lte(0.55) and ndvi_img.gte(0.1))
  for band in ['sur_refl_b01','sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07']:
    hist=sr_img.select(band).reduceRegion(reducer=ee.Reducer.fixedHistogram(min=-100,max=16000,steps=32),scale=500,bestEffort=True,geometry=region.geometry())
    hist_data=hist.get(band).getInfo()
    data=np.asarray(hist_data)
    if hist_data is None:
      flag=1
      break
    if band == 'sur_refl_b01':
        hist2d=data[:,1]
        hist2d=np.expand_dims(hist2d,axis=1)
    else:
        hist2d=np.concatenate((hist2d,np.expand_dims(data[:,1],axis=1)),axis=1)
    #print(hist3d.shape)
  if flag:
    flag=0
    continue
  for band in ['LST_Day_1km','LST_Night_1km']:
    hist=lst_img.select(band).reduceRegion(reducer=ee.Reducer.fixedHistogram(min=7500,max=65535,steps=32),scale=0.02,bestEffort=True,geometry=region.geometry())
    hist_data=hist.get(band).getInfo()
    data=np.asarray(hist_data)
    hist2d=np.concatenate((hist2d,np.expand_dims(data[:,1],axis=1)),axis=1)
    #print(hist3d.shape)
  hist3d.append(hist2d)
  print(len(hist3d))
hist3d=np.asarray(hist3d)
hist3d=np.swapaxes(hist3d,0,1)
print(hist3d.shape)

46
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
